In [378]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article
import re
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import pandas as pd
data = pd.read_csv('vocab.csv')
stopwords_ru=list(data['0'])
# символы для удаления
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [379]:
# лемматизация
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
#     print("len=",len(doc.split()))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    raznost=len(doc.split())-len(tokens)
#     print("/////////////////////////////////////")
#     print(raznost)
#     print(round(0.07*len(doc.split())))
#     print()
    if  raznost >= round(0.1*len(doc.split())):
#         print(0)
#         return tokens
        return 0
    else:
#         print(112)
        return 112
    
#     return None

In [380]:
# security lab 
# выясняем сколько страниц на сайие
url="https://www.securitylab.ru/news/page1_1.php"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
result = requests.get(url, headers=headers)
# поиск по тегам
soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
box = soup.find('div', class_='page-picker')
st = str(box.find('input'))
num_page=''
for i in range(12,len(st)):
    if st[i]=='"':
        break
    else:
        num_page=num_page+st[i]
num_page=int(num_page)
# num_page=2

In [381]:
# # выясняем сколько страниц на сайие
# url="https://www.securitylab.ru/news/page1_1.php"
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# result = requests.get(url, headers=headers)
# # поиск по тегам
# soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
# box = soup.find('div', class_='page-picker')
# st = str(box.find('input'))
# num_page=int(box)
# links = [link['href'] for link in box.find_all(href=True)]
# num_page=''
# for i in range (6, len(links[1])):
#     if links[1][i]=='&':
#         break
#     else:
#         num_page=num_page+links[1][i]
# num_page=int(num_page)

In [382]:
# получить контент
def get_news(url, data_):
    news = Article(url, language='ru')
    news.download()
    news.parse()
    data_frame=pd.DataFrame()
    el=str(news.text)
    text = copy.deepcopy(el[el.find("\n",el.find("\n")+2)+2:].lower())
    data_frame=pd.DataFrame()
    data_frame=pd.concat([data_frame,pd.DataFrame([text])])
    flag=data_frame[0].apply(lemmatize)
    if flag[0]==0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[1],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
#     else:
#         data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[0],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
    return data_
# 0 - текст новости
# 1 - относится ли к ИБ (1-да, 0- нет)
# 2 - является фейком (1-да,0 - нет)
# 3 - ссылка на новость

In [384]:
# собираем сслки на статьи на странице каждой
dataframe=pd.DataFrame()
for k in range(1,num_page+1):
    url="https://www.securitylab.ru/news/page1_"+str(k)+".php"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find_all('a', class_='article-card inline-card')
    link=[]
    for i in range (0, len(box)):
        try:
            stroka_url=str(box[i])
            urls=''
            for j in range (42,len(stroka_url)):
                if stroka_url[j]!='"':
                    urls=urls+stroka_url[j]
                else:
                    dataframe=get_news("https://www.securitylab.ru"+urls,dataframe)
                    break
            if len(dataframe)>=20:
                break
        except ValueError:
            e=1
    if len(dataframe)>=20:
            break
    print(len(dataframe)," k=",k)
        
dataframe

1  k= 1
3  k= 2
4  k= 3
5  k= 4
6  k= 5
10  k= 6
13  k= 7
15  k= 8


,News,InfSec,FAKE?,Link
0,В OAS Platform обнаружены восемь уязвимостей\n...,1,0,https://www.securitylab.ru/news/531871.php
0,Деньги – вот основной мотив современных хакеро...,1,0,https://www.securitylab.ru/news/531856.php
0,Уязвимость Cisco ASA могла позволить злоумышле...,1,0,https://www.securitylab.ru/news/531855.php
0,Staffcop Enterprise – информационная безопасно...,1,0,https://www.securitylab.ru/news/531834.php
0,DLP-система SecureTower v6.6: основное о новой...,1,0,https://www.securitylab.ru/news/531815.php
0,С февраля в Сеть утекли данные 8 млн клиентов ...,1,0,https://www.securitylab.ru/news/531804.php
0,Искусственный интеллект уничтожил вредоносное ...,1,0,https://www.securitylab.ru/news/531793.php
0,Активность Linux-вредоноса XorDDos выросла на ...,1,0,https://www.securitylab.ru/news/531791.php
0,Генпрокурор Великобритании: международное прав...,1,0,https://www.securitylab.ru/news/531780.php
0,Кибератака привела к сбоям в системе здравоохр...,1,0,https://www.securitylab.ru/news/531778.php


In [ ]:
# # собираем сслки на статьи на странице каждой
# dataframe=pd.DataFrame()
# for k in range(1,num_page+1):
#     url="https://digital.gov.ru/ru/events/monitoring/?words=&type=&directions=&start_date=&end_date=&page="+str(k)
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
#     result = requests.get(url, headers=headers)
#     soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
#     box = soup.find_all('li', class_='')
#     link=[]
#     for i in range (0, len(box)):
#         try:
#             links = [link['href'] for link in box[i].find_all(href=True)]
#             if links[0][0:11]=='/ru/events/' and len(links[0])>=11 and int(links[0][11]):
#                 get_news('https://digital.gov.ru'+links[0])
# #                 link.append(copy.deepcopy(links[0]))
#         except ValueError:
#             e=1
# #     print(link)

In [ ]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# html_page = requests.get('https://digital.gov.ru/ru/events/41570/',headers=headers).content
# soup = BeautifulSoup(html_page, 'lxml')
# whitelist = ['h1', 'h2', 'h3','strong','b','u','em','i','ins','p','br','hr']
# out = ""

# for t in soup.find_all(text=True):
#     if t.parent.name in whitelist:
#         out += '{} '.format(t)

# escape = ['\r', '\n', '\t', '\xa0']
# for e in escape:
#     out = out.replace(e, '')
# out

In [136]:
# url="https://digital.gov.ru/ru/events/41568/"
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
# box = soup.find_all('div', class_='col-lg-8 col-md-8 text-content')
# link=[]
# for i in range (0, len(box)):
#     print(box[i].find_all('p'))
#     print()
#     links = [link['p'] for link in box[i].find_all('p')]

In [148]:
# data = pd.read_csv('vocab.csv')
# list(data['0'])

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

driver.get('https://rg.ru/search?text=%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C&sort=date')

button = driver.find_element(By.CLASS_NAME, 'LoadMoreBtn_wrapper__8OS4F')
button.click()

In [23]:
round(4.25)

4